<a href="https://colab.research.google.com/github/16Komang/Big-Data-Tugas-7/blob/main/movie_recommendations_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.system("rm -rf /content/ml-predictions.csv")
os.system("rm -rf /content/mymodel")

0

In [4]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=bd461f476f85c26d8a808d56b1558e0c3826411355fb32866779f322f1132865
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [6]:
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Read the ratings data from a file
lines = spark.read.text("/content/drive/MyDrive/Big data/ml-1m/ratings.dat").rdd

# Split each line into fields and create a Row object
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]), rating=float(p[2]), timestamp=int(p[3])))

# Create a DataFrame from the RDD
ratings = spark.createDataFrame(ratingsRDD)

In [7]:
# Split the data into training and test sets
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
predictions.show()

# Calculate the RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("RMSE: ", rmse)

+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|   148|     50|   2.0|979577217| 4.1688976|
|   148|     95|   3.0|977334026| 3.6745012|
|   148|    107|   4.0|979577717| 3.3476396|
|   148|    158|   5.0|979578290| 3.3979707|
|   148|    160|   2.0|977354338| 3.0817404|
|   148|    165|   3.0|977333786|  4.105658|
|   148|    173|   3.0|977334225| 3.2034583|
|   148|    181|   1.0|977334454| 2.9556699|
|   148|    257|   4.0|979578268| 3.3077576|
|   148|    260|   5.0|977333254| 4.5606194|
|   148|    292|   4.0|977333869| 4.0279317|
|   148|    327|   1.0|977334427| 3.3208263|
|   148|    332|   3.0|979578850| 3.6227026|
|   148|    351|   3.0|977335047|  3.381733|
|   148|    370|   4.0|979578765| 3.3040617|
|   148|    377|   4.0|977333369|  4.272643|
|   148|    379|   5.0|977333890| 3.9711761|
|   148|    405|   2.0|977334427| 3.1063077|
|   148|    434|   4.0|977333974|  4.013489|
|   148|  

In [8]:
# Calculate the MSE
mse = predictions.rdd.map(lambda row: (row['rating'] - row['prediction'])**2).mean()
print("MSE: ", mse)

MSE:  0.8050899532714969
